In [17]:
import pandas as pd
import numpy as np

# Load the dataset (update the file name if needed)
df = pd.read_csv(r"C:\Users\thapa\OneDrive\Desktop\Retail-Insights_Project\data\superstore_raw.csv",
                 encoding='latin1')



print(df.head())
print(df.info())


   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
2       3  CA-2016-138688   6/12/2016   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2  Darrin Van Huff  Corporate  United States      Los Angeles  ...   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale  ...   

  Postal Code  Region       Product ID         Category Sub-Category  \
0       42420   Sout

In [18]:
# Clean column names
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('-', '_')


In [19]:
date_cols = ['Order_Date', 'Ship_Date']

for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')


In [20]:
# Remove duplicates
df.drop_duplicates(inplace=True)

# Remove rows with negative sales (if any)
df = df[df['Sales'] >= 0]

# Replace missing values where possible
df['Postal_Code'] = df['Postal_Code'].fillna(0)


In [21]:
float_cols = ['Sales', 'Profit', 'Discount']
int_cols = ['Quantity', 'Postal_Code']

for col in float_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

for col in int_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce', downcast='integer')


In [22]:
cat_cols = ['Ship_Mode', 'Segment', 'Country', 'City', 'State',
            'Region', 'Category', 'Sub_Category']

for col in cat_cols:
    df[col] = df[col].astype(str).str.strip()


In [23]:
# Latest date in dataset
max_date = df['Order_Date'].max()

# RFM calculation
rfm = df.groupby('Customer_ID').agg({
    'Order_Date': lambda x: (max_date - x.max()).days,
    'Order_ID': 'count',
    'Sales': 'sum'
}).reset_index()

rfm.columns = ['Customer_ID', 'Recency', 'Frequency', 'Monetary']

# Merge RFM into main dataframe for convenience
df = df.merge(rfm, on='Customer_ID', how='left')


In [24]:
df['Order_Year'] = df['Order_Date'].dt.year
df['Order_Month'] = df['Order_Date'].dt.month
df['Order_Month_Name'] = df['Order_Date'].dt.strftime('%B')


In [25]:
df.to_csv("../data/superstore_clean.csv", index=False)

print("CLEAN DATA SHAPE:", df.shape)


CLEAN DATA SHAPE: (9994, 27)


In [26]:
df.columns.tolist()


['Row_ID',
 'Order_ID',
 'Order_Date',
 'Ship_Date',
 'Ship_Mode',
 'Customer_ID',
 'Customer_Name',
 'Segment',
 'Country',
 'City',
 'State',
 'Postal_Code',
 'Region',
 'Product_ID',
 'Category',
 'Sub_Category',
 'Product_Name',
 'Sales',
 'Quantity',
 'Discount',
 'Profit',
 'Recency',
 'Frequency',
 'Monetary',
 'Order_Year',
 'Order_Month',
 'Order_Month_Name']

In [29]:
import pyodbc
pyodbc.drivers()


['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']

In [34]:
import pyodbc
import pandas as pd

# Load your cleaned CSV
df = pd.read_csv(
    r"C:\Users\thapa\OneDrive\Desktop\Retail-Insights_Project\data\superstore_clean.csv"
)

# Azure SQL connection details
server = 'tcp:retail-sql-server-007.database.windows.net'
database = 'RetailInsightsDB'
username = 'admin_milind'
password = 'Hustle9769!'

# Use the default driver available on your system
conn_str = (
    f"DRIVER={{SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password}"
)

# Connect to Azure SQL
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Insert rows
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO superstore_orders (
            Row_ID, Order_ID, Order_Date, Ship_Date, Ship_Mode, Customer_ID,
            Customer_Name, Segment, Country, City, State, Postal_Code,
            Region, Product_ID, Category, Sub_Category, Product_Name,
            Sales, Quantity, Discount, Profit, Recency, Frequency,
            Monetary, Order_Year, Order_Month, Order_Month_Name
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """,
    row.Row_ID, row.Order_ID, row.Order_Date, row.Ship_Date, row.Ship_Mode, row.Customer_ID,
    row.Customer_Name, row.Segment, row.Country, row.City, row.State, row.Postal_Code,
    row.Region, row.Product_ID, row.Category, row.Sub_Category, row.Product_Name,
    row.Sales, row.Quantity, row.Discount, row.Profit, row.Recency, row.Frequency,
    row.Monetary, row.Order_Year, row.Order_Month, row.Order_Month_Name)

conn.commit()
cursor.close()
conn.close()

print("Upload complete!")


Upload complete!
